<a href="https://colab.research.google.com/github/sdding/wanted_pre_onboarding/blob/main/%EC%9B%90%ED%8B%B0%EB%93%9C_%ED%94%84%EB%A6%AC%EC%98%A8%EB%B3%B4%EB%94%A9_%EC%BD%94%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import numpy as np
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [33]:
class Tokenizer():
  def __init__(self):
    self.word_dict = {'oov': 0}
    self.fit_checker = False

  def preprocessing(self, sequences):
    result= []
    for text in sequences:
      result.append(text_to_word_sequence(text.lower()))
    
    return result
  
  def fit(self, sequences):
    self.fit_checker = False
    tokens = self.preprocessing(sequences)
    
    i = 1
    for token in tokens:
      for t in token:
        if t not in self.word_dict:
          self.word_dict[t] = i
          i += 1

    self.fit_checker = True
    
  def transform(self, sequences):
    tokens = self.preprocessing(sequences)

    result = []
    if self.fit_checker:
      for token in tokens:
        L = []
        for t in token:
          if t not in self.word_dict:   # 어휘 사전에 없는 단어는 'oov'의 index 0 으로 변환
            L.append(0)
          else:
            L.append(self.word_dict[t])
        result.append(L)
          
      return result
    else:
      raise Exception("Tokenizer instance is not fitted yet.")
      
  def fit_transform(self, sequences):
    self.fit(sequences)
    result = self.transform(sequences)
    return result


In [34]:
Tokenizer().fit_transform(['I go to school.', 'I LIKE pizza!'])

[[1, 2, 3, 4], [1, 5, 6]]

In [35]:
class TfidfVectorizer:
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer
    self.fit_checker = False
  
  def fit(self, sequences):
    tokenized = self.tokenizer.fit_transform(sequences)

    M = max(map(max, tokenized))   # 토큰 숫자중 최대값
    IDF = []
    
    for i in range(1, M+1):
      count = 0
      for tokens in tokenized:
        if i in tokens:
          count += 1
      IDF.append(np.log(len(tokenized) / (1+count)))  # IDF 값
    
    self.fit_checker = True
    return IDF

  def transform(self, sequences):
    if self.fit_checker:
      tokenized = self.Tokenizer().transform(sequences)

      self.IDF = self.fit()     # IDF 리스트 받아오기
      M = max(map(max, tokenized))
      TF = []
      for tokens in tokenized:
        count = []
        for i in range(1, M+1):
          count.append(tokens.count(i))
        TF.append(count)
        
      tfidf_matrix = np.dot(TF, self.IDF)

      return self.tfidf_matrix
    else:
      raise Exception("TfidfVectorizer instance is not fitted yet.")

  
  def fit_transform(self, sequences):
    self.fit(sequences)
    return self.transform(sequences)

In [50]:
TfidfVectorizer(Tokenizer()).fit_transform(['I go to school.', 'I LIKE pizza!'])

AttributeError: ignored

In [38]:
tokenized = Tokenizer().fit_transform(['I go to school.', 'I LIKE pizza!'])
M = max(map(max, tokenized))   # 토큰 숫자중 최대값
IDF = []
i = 1
while i <= M:
  count = 0
  for tokens in tokenized:
    if i in tokens:
      count += 1
  IDF.append(np.log(len(tokenized) / (1+count)))
  i += 1 
print(IDF)


TF = []
      
for tokens in tokenized:
  count = []
  for i in range(1, M+1):
    count.append(tokens.count(i))
  TF.append(count)
print(TF)


tfidf_matrix = np.dot(TF, IDF)
print(tfidf_matrix)

[-0.40546510810816444, 0.0, 0.0, 0.0, 0.0, 0.0]
[[1, 1, 1, 1, 0, 0], [1, 0, 0, 0, 1, 1]]
[-0.40546511 -0.40546511]
